In [20]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from pickle import load
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import TimeDistributed


In [21]:
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy, huber_loss

In [3]:
import os
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r",encoding="utf8") as f:
        data = f.read().splitlines()

    return data


In [4]:
english_sentences = load_data('tel_eng.txt')
french_sentences = load_data('tel_tel.txt')
print('Dataset Loaded')

Dataset Loaded


In [5]:
type(english_sentences)

list

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  After Independence, the Indian government taken police action upon the Nizam's kingdom then they fromed these places as a Hyderabad state.
small_vocab_fr Line 1:  స్వాతంత్ర్యం తరువాత భారత ప్రభుత్వం నిజాము సంస్థానంపై జరిపిన పోలీసు చర్య తరువాత ఈ ప్రాంతాలు హైదరాబాదు రాష్ట్రంగా ఏర్పడ్డాయి.
small_vocab_en Line 2:  Michael Jones Jackson(Birth August 29 1958)(Death June 25 2009)a musical artist belonged to America.
small_vocab_fr Line 2:  మైకల్ జోసెఫ్ జాక్సన్ (జననం ఆగష్టు 29 1958)(మరణం జూన్ 25 2009) అమెరికాకు చెందిన ఒక ప్రముఖ సంగీత కళాకారుడు.


In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

10467 English words.
4451 unique English words.
10 Most common words in the English dataset:
"the" "of" "in" "is" "and" "to" "are" "as" "this" "was"

9290 French words.
5115 unique French words.
10 Most common words in the French dataset:
"ఈ" ":" "-" "విశాఖపట్నం" "అమెరికా" "ఒక" "స్థాపన" "ఉన్న" "విశాఖ" "నుంచి"


In [8]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [9]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [10]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 57
Max French sentence length: 52
English vocabulary size: 3391
French vocabulary size: 4720


In [11]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')


`logits_to_text` function loaded.


In [28]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(4720))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
8/8 [==============================] - 11s 1s/step - loss: 177.3801 - accuracy: 0.4879 - val_loss: 254.0947 - val_accuracy: 0.5030
Epoch 2/10
8/8 [==============================] - 8s 1s/step - loss: 177.3801 - accuracy: 0.4875 - val_loss: 254.0947 - val_accuracy: 0.5025
Epoch 3/10
8/8 [==============================] - 8s 1s/step - loss: 177.3801 - accuracy: 0.4871 - val_loss: 254.0947 - val_accuracy: 0.5020
Epoch 4/10
8/8 [==============================] - 8s 1s/step - loss: 177.3801 - accuracy: 0.4862 - val_loss: 254.0947 - val_accuracy: 0.5009
Epoch 5/10
8/8 [==============================] - 8s 1000ms/step - loss: 177.3801 - accuracy: 0.4851 - val_loss: 254.0947 - val_accuracy: 0.4998
Epoch 6/10
8/8 [==============================] - 8s 1s/step - loss: 177.3801 - accuracy: 0.4840 - val_loss: 254.0947 - val_accuracy: 0.4989
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 177.3801 - accuracy: 0.4826 - val_loss: 254.0947 - val_accuracy: 0.4983
Epoch 8/

In [27]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=huber_loss,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1/1 [==============================] - 40s 40s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 21s 21s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 22s 22s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 24s 24s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 23s 23s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 34s 34s/step - loss: 177.3801 - accuracy: 5.7121e-05 - val_loss: 254.0947 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 26s 26s/step - loss: 177.3801 - accuracy: 5.

In [29]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
1/1 [==============================] - 26s 26s/step - loss: 177.3801 - accuracy: 0.4931 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 2/20
1/1 [==============================] - 22s 22s/step - loss: 177.3801 - accuracy: 0.4934 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 3/20
1/1 [==============================] - 19s 19s/step - loss: 177.3801 - accuracy: 0.4956 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 4/20
1/1 [==============================] - 22s 22s/step - loss: 177.3801 - accuracy: 0.4944 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 5/20
1/1 [==============================] - 23s 23s/step - loss: 177.3801 - accuracy: 0.4929 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 6/20
1/1 [==============================] - 20s 20s/step - loss: 177.3801 - accuracy: 0.4958 - val_loss: 254.0947 - val_accuracy: 0.5052
Epoch 7/20
1/1 [==============================] - 20s 20s/step - loss: 177.3801 - accuracy: 0.4949 - val_loss: 254.0947 - val_accuracy: 0.5052

In [32]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
1/1 [==============================] - 28s 28s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 2/20
1/1 [==============================] - 19s 19s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 3/20
1/1 [==============================] - 18s 18s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 4/20
1/1 [==============================] - 18s 18s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 5/20
1/1 [==============================] - 17s 17s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 6/20
1/1 [==============================] - 17s 17s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329
Epoch 7/20
1/1 [==============================] - 17s 17s/step - loss: 177.3801 - accuracy: 0.8286 - val_loss: 254.0947 - val_accuracy: 0.8329

In [12]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
print('Final Model Loaded')

Final Model Loaded


In [22]:
def final_predictions(x, y, x_tk, y_tk):
    global predictions
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
    
final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/17
1/1 [==============================] - 79s 79s/step - loss: 8.4651 - accuracy: 3.8081e-05 - val_loss: 8.1150 - val_accuracy: 0.8608
Epoch 2/17
1/1 [==============================] - 58s 58s/step - loss: 8.1185 - accuracy: 0.8546 - val_loss: 3.5907 - val_accuracy: 0.8608
Epoch 3/17
1/1 [==============================] - 53s 53s/step - loss: 3.6123 - accuracy: 0.8546 - val_loss: 1.7595 - val_accuracy: 0.8608
Epoch 4/17
1/1 [==============================] - 47s 47s/step - loss: 1.8339 - accuracy: 0.8546 - val_loss: 1.7806 - val_accuracy: 0.8608
Epoch 5/17
1/1 [==============================] - 57s 57s/step - loss: 1.8175 - accuracy: 0.8546 - val_loss: 1.5394 - val_accuracy: 0.8608
Epoch 6/17
1/1 [==============================] - 52s 52s/step - loss: 1.5212 - accuracy: 0.8546 - val_loss: 1.7917 - val_accuracy: 0.8608
Epoch 7/17
1/1 [==============================] - 42s 42s/step - loss: 1.7259 - accuracy: 0.8546 - val_loss: 1.4864 - val_accuracy: 0.8608
Epoch 8/17
1/1 [=======